The purpose of the script is to calculate GCPs and georeference jpg files. The source of the jpg files is the Central Geological Database: https://geolog.pgi.gov.pl/. One jpg file was georeferenced manually using the QGIS program. The remaining GCP points were automatically calculated and georeferencing was performed on their basis. The georeferenced tif files were then clipped to the area calculated from the GCPs.

In [ ]:
import pandas as pd
import numpy as np
import os
from osgeo import osr, ogr
import time
!pip install gdal
from osgeo import gdal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Input:
1. File txt with GCP according EPSG4326.
2. Seven jpg files without georeference.

In [ ]:
# os.mkdir("/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef ")
# !ls

In [ ]:
os.chdir("/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef ")
!ls

 coordinates_WGS84.txt	'Kopia mgsp2A0003.jpg'	'Kopia mgsp2A0006.jpg'	  'Kopia wspolrzedne.txt'
'Kopia mgsp2A0001.jpg'	'Kopia mgsp2A0004.jpg'	'Kopia mgsp2A0007.jpg'
'Kopia mgsp2A0002.jpg'	'Kopia mgsp2A0005.jpg'	'Kopia wspolrzedne1.txt'


In [ ]:
src_Path = '/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef '
input_files_without_geo = [r for r in os.listdir(src_Path) if r.endswith('jpg')]
input_files_without_geo

['Kopia mgsp2A0007.jpg',
 'Kopia mgsp2A0006.jpg',
 'Kopia mgsp2A0001.jpg',
 'Kopia mgsp2A0005.jpg',
 'Kopia mgsp2A0003.jpg',
 'Kopia mgsp2A0002.jpg',
 'Kopia mgsp2A0004.jpg']

In [ ]:
# Sort files based on the numeric part at the end of each filename
input_7files_without_geo = sorted(input_files_without_geo, key=lambda x: int(x.split('A')[1].split('.')[0]))
input_7files_without_geo

['Kopia mgsp2A0001.jpg',
 'Kopia mgsp2A0002.jpg',
 'Kopia mgsp2A0003.jpg',
 'Kopia mgsp2A0004.jpg',
 'Kopia mgsp2A0005.jpg',
 'Kopia mgsp2A0006.jpg',
 'Kopia mgsp2A0007.jpg']

Output:
1. Seven tif files with georeference.

In [ ]:
output_7files_georeferenced = []
for srcRst in input_7files_without_geo:
  dstRst = 'georef_' + srcRst[6:-4] + '.tif'
  output_7files_georeferenced.append(dstRst)
output_7files_georeferenced

['georef_mgsp2A0001.tif',
 'georef_mgsp2A0002.tif',
 'georef_mgsp2A0003.tif',
 'georef_mgsp2A0004.tif',
 'georef_mgsp2A0005.tif',
 'georef_mgsp2A0006.tif',
 'georef_mgsp2A0007.tif']

Input 7 tif files after georeference for clipping

In [ ]:
output_7files_clipped = []
for srcRst in input_7files_without_geo:
  dstRst = 'clip_' + srcRst[6:-4] + '.tif'
  output_7files_clipped.append(dstRst)
output_7files_clipped

In [ ]:
input_7files_with_geo1 = [r for r in os.listdir(src_Path) if r.startswith('temp')]
input_7files_with_geo1

['temp_georef_0007.tif',
 'temp_georef_0001.tif',
 'temp_georef_0002.tif',
 'temp_georef_0003.tif',
 'temp_georef_0004.tif',
 'temp_georef_0005.tif',
 'temp_georef_0006.tif']

In [ ]:
# Sort files based on the numeric part at the end of each filename
input_7files_with_geo = sorted(input_7files_with_geo1, key=lambda x: int(x.split('_')[-1].split('.')[0]))
input_7files_with_geo

['temp_georef_0001.tif',
 'temp_georef_0002.tif',
 'temp_georef_0003.tif',
 'temp_georef_0004.tif',
 'temp_georef_0005.tif',
 'temp_georef_0006.tif',
 'temp_georef_0007.tif']

In [ ]:
file_path = '/content/drive/MyDrive/Dane z Kaggla/GDAL/Raster/JPG/10JPG_without_georef /coordinates_WGS84.txt'

Calculating GCP.

In [ ]:
class CalculatingGCPs:
    def __init__(self, gcp_txt_path, num_repetitions=6, increment_value=0.25):
        self.gcp_txt_path = gcp_txt_path
        self.num_repetitions = num_repetitions
        self.increment_value = increment_value

    def creating_gcp1(self, path):
        data = pd.read_csv(path, header=None)
        col = data[[0, 1, 2, 3]]
        df = col[1:]
        df[[0, 1, 2, 3]] = df[[0, 1, 2, 3]].astype(float)
        df[3] = df[3].abs()
        return df

    def creating_gcp2(self, df, num_repetitions, increment_value):
        for _ in range(num_repetitions):
            df_last4_modified = df.iloc[-4:].copy()
            df_last4_modified[0] = df_last4_modified[0] + increment_value
            df = pd.concat([df, df_last4_modified], ignore_index=True)
        return df

    def creating_gcp3(self, df):
        list_of_lists_of_tuples = []
        for start in range(0, len(df), 4):
            chunk = df.iloc[start:start + 4]
            list_of_tuples = [tuple(row) for row in chunk.values]
            list_of_lists_of_tuples.append(list_of_tuples)
        return list_of_lists_of_tuples

    def process(self):
        initial_df = self.creating_gcp1(self.gcp_txt_path)
        extended_df = self.creating_gcp2(initial_df, self.num_repetitions, self.increment_value)
        gcps_list = self.creating_gcp3(extended_df)
        return gcps_list




In [ ]:
# Example usage
gcp_txt_path = file_path
calc_gcps = CalculatingGCPs(gcp_txt_path)
gcps_list = calc_gcps.process()

# for i, gcps in enumerate(gcps_list):
#     print(f"GCP set {i + 1}:")
#     for gcp in gcps:
#         print(gcp)

<ipython-input-56-0ad53f1d5d19>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[0, 1, 2, 3]] = df[[0, 1, 2, 3]].astype(float)
<ipython-input-56-0ad53f1d5d19>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[3] = df[3].abs()


In [ ]:
gcps_list

[[(16.999505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.248736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.24963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.000408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.249505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.498736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.49963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.250408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.499505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.748736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.74963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.5686274509

Georeferencja

In [ ]:
class GeoReference:
    def __init__(self, input_jpegs, output_tiffs, gcp_txt_path, srs_epsg, num_repetitions=6, increment_value=0.25):
        self.input_jpegs = input_jpegs
        self.output_tiffs = output_tiffs
        self.gcp_txt_path = gcp_txt_path
        self.srs_epsg = srs_epsg
        self.num_repetitions = num_repetitions
        self.increment_value = increment_value

    def creating_gcp1(self, path):
        data = pd.read_csv(path, header=None)
        col = data[[0, 1, 2, 3]]
        df = col[1:]
        df[[0, 1, 2, 3]] = df[[0, 1, 2, 3]].astype(float)
        df[3] = df[3].abs()
        return df

    def creating_gcp2(self, df, num_repetitions, increment_value):
        for _ in range(num_repetitions):
            df_last4_modified = df.iloc[-4:].copy()
            df_last4_modified[0] = df_last4_modified[0] + increment_value
            df = pd.concat([df, df_last4_modified], ignore_index=True)
        return df

    def creating_gcp3(self, df):
        list_of_lists_of_tuples = []
        for start in range(0, len(df), 4):
            chunk = df.iloc[start:start + 4]
            list_of_tuples = [tuple(row) for row in chunk.values]
            list_of_lists_of_tuples.append(list_of_tuples)
        return list_of_lists_of_tuples

    def process_gcp(self):
        initial_df = self.creating_gcp1(self.gcp_txt_path)
        extended_df = self.creating_gcp2(initial_df, self.num_repetitions, self.increment_value)
        gcps_list = self.creating_gcp3(extended_df)
        return gcps_list

    @staticmethod
    def create_gdal_gcps(gcp_tuples_list):
        gdal_gcps_list = []
        for gcp_tuples in gcp_tuples_list:
            gdal_gcps = [gdal.GCP(x, y, 0, pixel, line) for x, y, pixel, line in gcp_tuples]
            gdal_gcps_list.append(gdal_gcps)
        return gdal_gcps_list


    @staticmethod
    def calculate_geotransform(gcps):
        """
        Calculate the geotransform based on the provided GCPs, ensuring zero rotation terms.
        """
        # Extract GCP coordinates
        src_points = [(gcp.GCPPixel, gcp.GCPLine) for gcp in gcps]
        dst_points = [(gcp.GCPX, gcp.GCPY) for gcp in gcps]

        # Calculate scale factors
        pixel_size_x = (dst_points[1][0] - dst_points[0][0]) / (src_points[1][0] - src_points[0][0])
        pixel_size_y = (dst_points[2][1] - dst_points[0][1]) / (src_points[2][1] - src_points[0][1])

        # Calculate translation terms
        top_left_x = dst_points[0][0] - src_points[0][0] * pixel_size_x
        top_left_y = dst_points[0][1] - src_points[0][1] * pixel_size_y

        # Geotransform with zero rotation terms
        geotransform = [top_left_x, pixel_size_x, 0, top_left_y, 0, pixel_size_y]

        return geotransform

    def georeference_jpeg(self, input_jpeg, output_tiff, gcps):
        # Open the input JPEG file
        jpeg_dataset = gdal.Open(input_jpeg)
        if jpeg_dataset is None:
            print(f"Could not open {input_jpeg}")
            return

        # Get the dimensions of the image
        cols = jpeg_dataset.RasterXSize
        rows = jpeg_dataset.RasterYSize
        bands = jpeg_dataset.RasterCount

        # Create the output GeoTIFF file
        driver = gdal.GetDriverByName('GTiff')
        temp_tiff = f'temp_{output_tiff}'  # Unique temporary file for each output
        tiff_dataset = driver.Create(temp_tiff, cols, rows, bands, gdal.GDT_Byte)
        if tiff_dataset is None:
            print(f"Could not create {temp_tiff}")
            return

        # Set the GCPs
        gdal_gcps = [gdal.GCP(x, y, 0, pixel, line) for x, y, pixel, line in gcps]
        tiff_dataset.SetGCPs(gdal_gcps, "")

        # Set the projection
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(self.srs_epsg)
        tiff_dataset.SetProjection(srs.ExportToWkt())

        # Calculate and set the geotransform
        geotransform = self.calculate_geotransform(gdal_gcps)
        tiff_dataset.SetGeoTransform(geotransform)

        # Write the image data to the GeoTIFF file
        for i in range(1, bands + 1):
            jpeg_band = jpeg_dataset.GetRasterBand(i)
            jpeg_array = jpeg_band.ReadAsArray()
            tiff_band = tiff_dataset.GetRasterBand(i)
            tiff_band.WriteArray(jpeg_array)
            tiff_band.FlushCache()
            tiff_band.SetNoDataValue(0)

        # Close the datasets
        jpeg_dataset = None
        tiff_dataset = None

        # Apply gdal.Warp to perform TPS transformation and nearest neighbor resampling
        warp_options = gdal.WarpOptions(
            format='GTiff',
            resampleAlg=gdal.GRA_NearestNeighbour,
            tps=True
        )

        gdal.Warp(destNameOrDestDS=output_tiff, srcDSOrSrcDSTab=temp_tiff, options=warp_options)

        print(f"Georeferenced file saved as {output_tiff}")

    def process_all(self):
        gcps_list = self.process_gcp()
        for input_jpeg, output_tiff, gcps in zip(self.input_jpegs, self.output_tiffs, gcps_list):
            self.georeference_jpeg(input_jpeg, output_tiff, gcps)

# Example usage
input_jpegs = input_7files_without_geo
output_tiffs = output_7files_georeferenced
gcp_txt_path = file_path
srs_epsg = 4326

georef = GeoReference(input_jpegs, output_tiffs, gcp_txt_path, srs_epsg)
georef.process_all()

<ipython-input-51-87de60d11558>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[0, 1, 2, 3]] = df[[0, 1, 2, 3]].astype(float)
<ipython-input-51-87de60d11558>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[3] = df[3].abs()


Georeferenced file saved as georef_0001.tif
Georeferenced file saved as georef_0002.tif
Georeferenced file saved as georef_0003.tif
Georeferenced file saved as georef_0004.tif
Georeferenced file saved as georef_0005.tif
Georeferenced file saved as georef_0006.tif
Georeferenced file saved as georef_0007.tif


Clipping.

In [ ]:
def gdalGCPS(gcps):
    return [gdal.GCP(float(x), float(y), 0, float(pixel), float(line)) for x, y, pixel, line in gcps]

def create_polygon_shapefile(gcps, shp_filename, srs_epsg):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    if driver is None:
        print("ESRI Shapefile driver not available.")
        return False

    datasource = driver.CreateDataSource(shp_filename)
    if datasource is None:
        print(f"Could not create {shp_filename}")
        return False

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(srs_epsg)

    layer = datasource.CreateLayer("polygon", srs, ogr.wkbPolygon)
    if layer is None:
        print("Layer creation failed.")
        return False

    ring = ogr.Geometry(ogr.wkbLinearRing)
    for gcp in gcps:
        ring.AddPoint(float(gcp[0]), float(gcp[1]))

    ring.CloseRings()
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)

    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(poly)
    layer.CreateFeature(feature)

    feature = None
    layer = None
    datasource = None

    print(f"Polygon shapefile {shp_filename} created.")
    return True

def clip_georeferenced_tiff(input_tiff, output_tiff, shp_filename):
    input_dataset = gdal.Open(input_tiff)
    if input_dataset is None:
        print(f"Could not open {input_tiff}")
        return False

    shapefile_dataset = ogr.Open(shp_filename)
    if shapefile_dataset is None:
        print(f"Could not open {shp_filename}")
        return False

    shapefile_dataset = None  # Close shapefile dataset

    warp_options = gdal.WarpOptions(
        format='GTiff',
        cutlineDSName=shp_filename,
        cropToCutline=True,
        dstNodata=0
    )

    result = gdal.Warp(destNameOrDestDS=output_tiff, srcDSOrSrcDSTab=input_tiff, options=warp_options)
    if result is None:
        print(f"Failed to clip {input_tiff} using {shp_filename}")
        return False
    else:
        print(f"Clipped file saved as {output_tiff}")

    result = None  # Close the result dataset
    input_dataset = None  # Close the input dataset
    return True

def wait_for_file(filepath, timeout=60):
    start_time = time.time()
    while not os.path.exists(filepath):
        if time.time() - start_time > timeout:
            print(f"Timeout: {filepath} not found after {timeout} seconds")
            return False
        time.sleep(0.5)
    return True

In [ ]:
# Example usage
input_tiffs = input_7files_with_geo
output_tiffs = output_7files_clipped

gcps_list = [[(16.999505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.248736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.24963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.000408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.249505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.498736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.49963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.250408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.499505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.748736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.74963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.500408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.749505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (17.998736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (17.99963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (17.750408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(17.999505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (18.248736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (18.24963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (18.000408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(18.249505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (18.498736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (18.49963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (18.250408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)],
 [(18.499505600237047,
   54.83496447794371,
   342.07563025210067,
   855.6708683473391),
  (18.748736161698034,
   54.833158459382396,
   2866.7927170868347,
   855.6708683473391),
  (18.74963917097869,
   54.66700475174174,
   2873.2997198879552,
   3780.568627450981),
  (18.500408609517702,
   54.66700475174174,
   342.07563025210084,
   3774.0616246498603)]]

srs_epsg = 4326

for input_tiff, output_tiff, gcps in zip(input_tiffs, output_tiffs, gcps_list):
    shp_filename = f"{os.path.splitext(output_tiff)[0]}.shp"
    if create_polygon_shapefile(gcps, shp_filename, srs_epsg):
        if wait_for_file(input_tiff):
            clip_georeferenced_tiff(input_tiff, output_tiff, shp_filename)
        else:
            print(f"Skipping clipping for {input_tiff} as the file was not available in time.")

Polygon shapefile clip_0001.shp created.
Clipped file saved as clip_0001.tif
Polygon shapefile clip_0002.shp created.
Clipped file saved as clip_0002.tif
Polygon shapefile clip_0003.shp created.
Clipped file saved as clip_0003.tif
Polygon shapefile clip_0004.shp created.
Clipped file saved as clip_0004.tif
Polygon shapefile clip_0005.shp created.
Clipped file saved as clip_0005.tif
Polygon shapefile clip_0006.shp created.
Clipped file saved as clip_0006.tif
Polygon shapefile clip_0007.shp created.
Clipped file saved as clip_0007.tif
